In [1]:
import torch
from torch.autograd import Variable

In [2]:
# import torch
# from torch.autograd import Variable

# x = Variable(2*torch.ones(2, 2), requires_grad=True)
# y = Variable(3*torch.ones(2, 2), requires_grad=True)
# opt = torch.optim.SGD([x], lr=0.01)

# y.requires_grad = False
# f1 = x*2 + 3*y
# f1.backward(torch.ones_like(f1))
# opt.step()
# y.requires_grad = True
# f2 = 4 * f1 + 5 * y
# f1.backward(torch.ones_like(f2))
# print(x.grad, y.grad)

In [6]:
import numpy as np
import time
import torch
import torch
import sys
sys.path.append('pytorch')
sys.path.append('pytorch/utils')

from pytorch.mem_transformer import *
from pytorch import data_utils

from experiment_utils.run_experiment import *
from experiment_utils.generate_data import *

## Variables

In [7]:
from sklearn.model_selection import ParameterGrid

TAG = '10tkn_len24_ext'

TASK_NAME = 'reverse'
TRAIN_SIZE = 1000
VAL_SIZE = 200
TEST_SIZE = 100


NUM_BATCHES = int(4e5)
BATCH_SIZE = 128
GENERATE_EVERY  = 10000
NUM_TOKENS = 10 + 2
SEQ_LEN = 24

#### Generate data

In [8]:
# # !mkdir data24
# np.random.seed(42)

# generator = copy_generator(batch_size=BATCH_SIZE, enc_seq_len=ENC_SEQ_LEN, dec_seq_len=DEC_SEQ_LEN, num_tokens=NUM_TOKENS)
# generate_data(generator, path=f'data{INPUT_LEN}', task_name=TASK_NAME, train_size=TRAIN_SIZE, test_size=TEST_SIZE, val_size=VAL_SIZE, batch_size=BATCH_SIZE)  

In [9]:
np.arange(10)[-1:]

array([9])

In [10]:
class data_loader:
    def __init__(self, mode, path='data', task_name=TASK_NAME, batch_size=32, tgt_len=24, device='cpu'):
        X = np.load(f'{path}/{task_name}_{mode}_X.npy')
        y = np.load(f'{path}/{task_name}_{mode}_y.npy')

        src = np.hstack((X, y))
        src, tgt = src[:, :-1], src[:, 1:]

        self.src = torch.Tensor(src).long()
        self.tgt = torch.Tensor(tgt).long()

        self.data_size = self.src.shape[0]
        self.data_ptr = 0

        self.batch_size = batch_size
        self.tgt_len = tgt_len
        self.device = device

    def __iter__(self):
        return next(self)

    def __next__(self):
        while(True):
            src = self.src[self.data_ptr: self.data_ptr+self.batch_size].to(self.device)
            tgt = self.tgt[self.data_ptr: self.data_ptr+self.batch_size].to(self.device)
            
            self.data_ptr = (self.data_ptr + self.batch_size) % self.data_size

            yield src.T, tgt.T, self.tgt_len
        

In [11]:
# path = "/home/ayd98/Desktop/MIPT/Memory transformer/data24/"
# gen = data_loader('train', path, batch_size=BATCH_SIZE)

### Run

In [12]:
# path = "/home/ayd98/Desktop/MIPT/Memory transformer/data24"
# tr_iter = data_loader('train', path=path, task_name=f'{TASK_NAME}', batch_size=BATCH_SIZE, tgt_len=SEQ_LEN)
# te_iter = data_loader('test', path=path, task_name=f'{TASK_NAME}', batch_size=BATCH_SIZE, tgt_len=SEQ_LEN)
# va_iter = data_loader('val', path=path, task_name=f'{TASK_NAME}', batch_size=BATCH_SIZE, tgt_len=SEQ_LEN)


from data_utils import get_lm_corpus
device='cpu'
batch_size = 2
tgt_len = eval_tgt_len = 51
ext_len = 0
corpus = get_lm_corpus('/home/ayd98/Desktop/MIPT/TXL/data/enwik8/', 'enwik8')
eval_batch_size = 10
tr_iter = corpus.get_iterator('train', batch_size, tgt_len,
    device=device, ext_len=ext_len)
va_iter = corpus.get_iterator('valid', eval_batch_size, eval_tgt_len,
    device=device, ext_len=ext_len)
te_iter = corpus.get_iterator('test', eval_batch_size, eval_tgt_len,
    device=device, ext_len=ext_len)

cutoffs, tie_projs = [], [False]
NUM_TOKENS = len(corpus.vocab)

Loading cached dataset...


In [13]:
model_parameters = ParameterGrid({
                'n_layer': [2],
                'n_head': [4],
                'd_head': [128],
                'num_mem_tokens': [9],
                'mem_len': [0], 
                'mem_at_end': [False]})

param = list(model_parameters)[0]

fixed_parameters = {'n_token': NUM_TOKENS,
                    'd_model': param['d_head'],# + param['num_mem_tokens']-1,
                    'd_inner': param['d_head'],
                    'dropout': 0,
                    'dropatt': 0,
                    'tie_weight': True,
                    'div_val': 1, # ???????
                    'tie_projs': [False],
                    'tgt_len': SEQ_LEN,
                    'ext_len': 0, 
                    'cutoffs': [],
                    'attn_type': 0,}

model = MemTransformerLM(**param, **fixed_parameters)#.cuda()


In [19]:
for i, n in enumerate(tr_iter):
    s, t, l = n
    if i == 1:
        break


In [67]:
mems = None

model._forward(s, mems)[0]

tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        ...,

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]],
       grad_fn=<NativeLayerNormBackward0>)

In [14]:
para_model = model
train_step = 0
train_loss = 0
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

mem_backprop_depth=1

In [17]:
print(model.layers[0].dec_attn.r_net.weight)

Parameter containing:
tensor([[ 0.0497,  0.0670, -0.0693,  ..., -0.0674, -0.0050,  0.0223],
        [ 0.0126, -0.0033, -0.0213,  ...,  0.0810,  0.0525, -0.0349],
        [ 0.0364, -0.0286, -0.0465,  ..., -0.0766,  0.0604, -0.0801],
        ...,
        [ 0.0398, -0.0857, -0.0380,  ...,  0.0082, -0.0561,  0.0282],
        [-0.0816,  0.0695, -0.0018,  ..., -0.0191,  0.0370, -0.0161],
        [-0.0596,  0.0635,  0.0120,  ..., -0.0694, -0.0774, -0.0256]],
       requires_grad=True)


In [15]:
for n,p in model.named_parameters():
    print(n)

mem_tokens
r_w_bias
r_r_bias
word_emb.emb_layers.0.weight
layers.0.dec_attn.qkv_net.weight
layers.0.dec_attn.o_net.weight
layers.0.dec_attn.layer_norm.weight
layers.0.dec_attn.layer_norm.bias
layers.0.dec_attn.r_net.weight
layers.0.pos_ff.CoreNet.0.weight
layers.0.pos_ff.CoreNet.0.bias
layers.0.pos_ff.CoreNet.3.weight
layers.0.pos_ff.CoreNet.3.bias
layers.0.pos_ff.layer_norm.weight
layers.0.pos_ff.layer_norm.bias
layers.1.dec_attn.qkv_net.weight
layers.1.dec_attn.o_net.weight
layers.1.dec_attn.layer_norm.weight
layers.1.dec_attn.layer_norm.bias
layers.1.dec_attn.r_net.weight
layers.1.pos_ff.CoreNet.0.weight
layers.1.pos_ff.CoreNet.0.bias
layers.1.pos_ff.CoreNet.3.weight
layers.1.pos_ff.CoreNet.3.bias
layers.1.pos_ff.layer_norm.weight
layers.1.pos_ff.layer_norm.bias
crit.out_layers.0.bias


In [ ]:

model.train()
mems = tuple()
prev_data, prev_target, prev_mems = [], [], []
for batch, (data, target, seq_len) in enumerate(tr_iter):
    if batch == 5000: 
        break
    model.zero_grad()

    if mem_backprop_depth > 0:
        prev_data = prev_data[-mem_backprop_depth:] + [data]
        prev_target = prev_target[-mem_backprop_depth:] + [target]
        prev_mems = prev_mems[-mem_backprop_depth:] + [mems]
        for pd, pt, pm in zip(prev_data, prev_target, prev_mems):
            # print(pd.shape, len(prev_data))
            ret = para_model(pd, pt, *pm)
    else:
        ret = para_model(data, target, *mems)
    loss, mems = ret[0], ret[1:]

    loss = loss.float().mean().type_as(loss)
    loss.backward()
    print(loss.item())
    train_loss += loss.float().item()

    optimizer.step()


In [92]:
for i, a in enumerate(gen_val):
    print(i, a[0].shape)

0 torch.Size([48, 128])
1 torch.Size([48, 128])
2 torch.Size([48, 128])
3 torch.Size([48, 128])
4 torch.Size([48, 128])
5 torch.Size([48, 128])
6 torch.Size([48, 128])
7 torch.Size([48, 128])
8 torch.Size([48, 128])
9 torch.Size([48, 128])
10 torch.Size([48, 128])
11 torch.Size([48, 128])
12 torch.Size([48, 128])
13 torch.Size([48, 128])
14 torch.Size([48, 128])
15 torch.Size([48, 80])
16 torch.Size([48, 128])
17 torch.Size([48, 128])
18 torch.Size([48, 128])
19 torch.Size([48, 128])
20 torch.Size([48, 128])
21 torch.Size([48, 128])
22 torch.Size([48, 128])
23 torch.Size([48, 128])
24 torch.Size([48, 128])
25 torch.Size([48, 128])
26 torch.Size([48, 128])
27 torch.Size([48, 128])
28 torch.Size([48, 128])
29 torch.Size([48, 128])
30 torch.Size([48, 128])
31 torch.Size([48, 32])
32 torch.Size([48, 128])
33 torch.Size([48, 128])
34 torch.Size([48, 128])
35 torch.Size([48, 128])
36 torch.Size([48, 128])
37 torch.Size([48, 128])
38 torch.Size([48, 128])
39 torch.Size([48, 128])
40 torch.Siz

KeyboardInterrupt: 

In [48]:
context_len = 16
n_segs = s.shape[0] / 16
[s]

tensor([[ 2,  2,  4,  ...,  9,  2, 11],
        [ 3,  3, 11,  ...,  5,  9, 11],
        [ 5,  9,  2,  ...,  5,  3,  7],
        ...,
        [11, 10, 10,  ...,  4,  7,  2],
        [ 5,  9,  2,  ...,  5,  3,  7],
        [ 3,  3, 11,  ...,  5,  9, 11]])

In [36]:
seq_len = 12
gen_train.src.shape

torch.Size([100000, 48])

In [34]:
(.max() - gen_train.src.min()).item()

10

In [32]:
ls "/home/ayd98/Desktop/MIPT/Memory transformer/data24"

copy_test_X.npy   copy_train_y.npy  reverse_test_X.npy   reverse_train_y.npy
copy_test_y.npy   copy_val_X.npy    reverse_test_y.npy   reverse_val_X.npy
copy_train_X.npy  copy_val_y.npy    reverse_train_X.npy  reverse_val_y.npy


In [29]:
for i, (src, tgt, l) in enumerate(gen_train):
    if i == 2:
        break

In [30]:
# mems = tuple()
# model(src, tgt, *mems)

In [31]:
# model.mem_tokens.shape

In [10]:
model.train()
mems = tuple()
mem_gradients = []

para_model = model
for batch, (data, target, seq_len) in enumerate(gen_train):
    model.zero_grad()

    ret = para_model(data, target, *mems)
    loss, mems = ret[0], ret[1:]

    loss = loss.float().mean().type_as(loss)
    if args.fp16:
        optimizer.backward(loss)
    else:
        loss.backward()
    train_loss += loss.float().item()

    if args.fp16:
        optimizer.clip_master_grads(args.clip)
    else:
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)

    if args.mem_backprop_depth > 0:
        mem_gradients = mem_gradients[-args.mem_backprop_depth:]
        mem_gradients.append(para_model.mem_tokens.grad.clone())
        new_grad = torch.stack(mem_gradients).sum(dim=0)
        para_model.mem_tokens.grad = new_grad

    optimizer.step()

    if args.sample_softmax > 0:
        optimizer_sparse.step()

    # step-wise learning rate annealing
    train_step += 1
    if args.scheduler in ['cosine', 'constant', 'dev_perf']:
        # linear warmup stage
        if train_step < args.warmup_step:
            curr_lr = args.lr * train_step / args.warmup_step
            optimizer.param_groups[0]['lr'] = curr_lr
            if args.sample_softmax > 0:
                optimizer_sparse.param_groups[0]['lr'] = curr_lr * 2
        else:
            if args.scheduler == 'cosine':
                scheduler.step(train_step)
                if args.sample_softmax > 0:
                    scheduler_sparse.step(train_step)
    elif args.scheduler == 'inv_sqrt':
        scheduler.step(train_step)

    if train_step % args.log_interval == 0:
        cur_loss = train_loss / args.log_interval
        elapsed = time.time() - log_start_time
        log_str = '| epoch {:3d} step {:>8d} | {:>6d} batches | lr {:.3g} ' \
                    '| ms/batch {:5.2f} | loss {:5.2f}'.format(
            epoch, train_step, batch+1, optimizer.param_groups[0]['lr'],
            elapsed * 1000 / args.log_interval, cur_loss)
        if args.dataset in ['enwik8', 'text8']:
            log_str += ' | bpc {:9.5f}'.format(cur_loss / math.log(2))
        else:
            log_str += ' | ppl {:9.3f}'.format(math.exp(cur_loss))
        logging(log_str)
        train_loss = 0
        log_start_time = time.time()

    if train_step % args.eval_interval == 0:
        val_loss = evaluate(va_iter)
        logging('-' * 100)
        log_str = '| Eval {:3d} at step {:>8d} | time: {:5.2f}s ' \
                    '| valid loss {:5.2f}'.format(
            train_step // args.eval_interval, train_step,
            (time.time() - eval_start_time), val_loss)
        if args.dataset in ['enwik8', 'text8']:
            log_str += ' | bpc {:9.5f}'.format(val_loss / math.log(2))
        else:
            log_str += ' | valid ppl {:9.3f}'.format(math.exp(val_loss))
        logging(log_str)
        logging('-' * 100)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            if not args.debug:
                with open(os.path.join(args.work_dir, 'model.pt'), 'wb') as f:
                    torch.save(model, f)
                with open(os.path.join(args.work_dir, 'optimizer.pt'), 'wb') as f:
                    torch.save(optimizer.state_dict(), f)
            best_val_loss = val_loss

        # dev-performance based learning rate annealing
        if args.scheduler == 'dev_perf':
            scheduler.step(val_loss)
            if args.sample_softmax > 0:
                scheduler_sparse.step(val_loss)

        eval_start_time = time.time()

    if train_step == args.max_step:
        break

SyntaxError: invalid syntax (<ipython-input-10-a870215ecc03>, line 6)

In [ ]:
gen_train = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_train', batch_size=BATCH_SIZE)
gen_val = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_val', batch_size=VAL_SIZE)
gen_test = data_loader(path=f'data{INPUT_LEN}', task_name=f'{TASK_NAME}_test', batch_size=TEST_SIZE)


print_file = f'logs/{TASK_NAME}_{TAG}_memory_logs.txt'
t = time.time()
with torch.cuda.device(0):
    for init_num in range(NUM_INITS):
        with open(print_file, 'a') as f:
            f.write('\n\nInit number ' + str(init_num)+'\n')
        for i, param in enumerate(list(model_parameters)):
            with open(print_file, 'a') as f:
                f.write('\n\n' + str(param)+'\n')
            param['enc_depth'], param['enc_heads'] = param['depth,heads']
            param['dec_depth'], param['dec_heads'] = param['depth,heads']
            param.pop('depth,heads')

            with open(print_file, 'a') as f:
                f.write(f'{i / len(model_parameters) * 100}%')
            model = TransformerXL
            model = XTransformer(**param).cuda()

            model_name = f"{TASK_NAME}{INPUT_LEN}_dim{param['dim']}d{param['enc_depth']}h{param['enc_heads']}M{param['enc_num_memory_tokens']}l{param['enc_max_seq_len']}_{TAG}_v{init_num}"

            optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
            train_validate_model(model, 
                            train_generator=gen_train, 
                            val_generator=gen_val, 
                            optim=optim, 
                            model_name=model_name, 
                            config=param,
                            num_batches=NUM_BATCHES,
                            generate_every=GENERATE_EVERY,
                            print_file=print_file,
                            tag=TAG,
                            overfit_stop=False)
            test_model(model, gen_test, model_name, param, TASK_NAME, tag=TAG)
            with open(print_file, 'a') as f:
                f.write(f'\nTotal time: {time.time() - t}\n')
            t = time.time()